##Import the Libaraies

In [43]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

##Import  the dataset

In [44]:
dataset=pd.read_csv("/content/train_u6lujuX_CVtuZ9i.csv")
X=dataset.iloc[:,:-1].values
y=dataset.iloc[:,-1]
print(dataset.columns)

Index(['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'Loan_Status'],
      dtype='object')


In [45]:
dataset.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


### Check for missing data

In [46]:
dataset.isnull().sum()

,0
Loan_ID,0
Gender,13
Married,3
Dependents,15
Education,0
Self_Employed,32
ApplicantIncome,0
CoapplicantIncome,0
LoanAmount,22
Loan_Amount_Term,14


#### Fill missing data for 'LoanAmount', 'Credit_History', etc.

In [58]:
dataset['LoanAmount'].fillna(dataset['LoanAmount'].median(), inplace=True)
dataset['Credit_History'].fillna(dataset['Credit_History'].mode()[0], inplace=True)
dataset['Dependents'].fillna(dataset['Dependents'].mode()[0], inplace=True)


<ipython-input-58-133ef4a9c673>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dataset['LoanAmount'].fillna(dataset['LoanAmount'].median(), inplace=True)
<ipython-input-58-133ef4a9c673>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value

## Encode categorical columns

In [48]:
le = LabelEncoder()
dataset['Gender'] = le.fit_transform(dataset['Gender'])
dataset['Married'] = le.fit_transform(dataset['Married'])
dataset['Education'] = le.fit_transform(dataset['Education'])
dataset['Self_Employed'] = le.fit_transform(dataset['Self_Employed'])
dataset['Property_Area'] = le.fit_transform(dataset['Property_Area'])
dataset['Dependents'] = le.fit_transform(dataset['Dependents'])

In [49]:
print(dataset.columns)

Index(['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'Loan_Status'],
      dtype='object')


#### Split into features and target variable

In [50]:
# Split into features and target variable
X = dataset.drop('Loan_Status', axis=1)
y = dataset['Loan_Status']

## Handling Missing Values

In [51]:
import pandas as pd
from sklearn.impute import SimpleImputer

# Load your training dataset
dataset = pd.read_csv('/content/train_u6lujuX_CVtuZ9i.csv')

# Separate features and target
X = dataset.drop(columns=['Loan_ID', 'Loan_Status'])  # Drop ID and target
y = dataset['Loan_Status']

# For numerical columns - fill missing values with median
num_cols = ['LoanAmount', 'Loan_Amount_Term', 'Credit_History']
num_imputer = SimpleImputer(strategy='median')
X[num_cols] = num_imputer.fit_transform(X[num_cols])

# For categorical columns - fill missing values with most frequent value
cat_cols = ['Gender', 'Married', 'Dependents', 'Self_Employed']
cat_imputer = SimpleImputer(strategy='most_frequent')
X[cat_cols] = cat_imputer.fit_transform(X[cat_cols])

# Check if missing values are handled
print(X.isnull().sum())  # All should print 0


Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
dtype: int64


### Encode Categorical Variables

In [52]:
from sklearn.preprocessing import LabelEncoder

# Create LabelEncoder instance
le = LabelEncoder()

# List of categorical columns
categorical_columns = ['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'Property_Area']

# Apply label encoding to each categorical column
for col in categorical_columns:
    X[col] = le.fit_transform(X[col])

# Encode the target as well: Loan_Status ('Y' -> 1, 'N' -> 0)
y = le.fit_transform(y)

# Check encoded data
print(X.head())
print("Target Labels:", y[:5])


   Gender  Married  Dependents  Education  Self_Employed  ApplicantIncome  \
0       1        0           0          0              0             5849   
1       1        1           1          0              0             4583   
2       1        1           0          0              1             3000   
3       1        1           0          1              0             2583   
4       1        0           0          0              0             6000   

   CoapplicantIncome  LoanAmount  Loan_Amount_Term  Credit_History  \
0                0.0       128.0             360.0             1.0   
1             1508.0       128.0             360.0             1.0   
2                0.0        66.0             360.0             1.0   
3             2358.0       120.0             360.0             1.0   
4                0.0       141.0             360.0             1.0   

   Property_Area  
0              2  
1              0  
2              2  
3              2  
4              2  
Ta

##Train the DecisionTreeClassifier

In [53]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, roc_auc_score

# Split data (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize Decision Tree model
model = DecisionTreeClassifier(random_state=42)

# Train the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# If binary labels (0 and 1), check ROC AUC
if len(set(y)) == 2:
    y_proba = model.predict_proba(X_test)[:, 1]
    auc = roc_auc_score(y_test, y_proba)
    print("AUC-ROC Score:", auc)

Accuracy: 0.6910569105691057
AUC-ROC Score: 0.6549418604651164


## Train the Random forest Classifier

In [54]:
from sklearn.ensemble import RandomForestClassifier

# Initialize Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
rf_model.fit(X_train, y_train)

# Make predictions
rf_pred = rf_model.predict(X_test)

# Evaluate model
rf_accuracy = accuracy_score(y_test, rf_pred)
print("Random Forest Accuracy:", rf_accuracy)

# AUC-ROC Score
rf_proba = rf_model.predict_proba(X_test)[:, 1]
rf_auc = roc_auc_score(y_test, rf_proba)
print("Random Forest AUC-ROC Score:", rf_auc)


Random Forest Accuracy: 0.7560975609756098
Random Forest AUC-ROC Score: 0.7552325581395348


###Save trained Random Forest model:

In [55]:
import joblib

# Save the Random Forest model
joblib.dump(rf_model, 'loan_model.pkl')


['loan_model.pkl']

###train the  XGBoost

In [56]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, roc_auc_score

# Train the XGBoost model
xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_model.fit(X_train, y_train)

# Predict
y_pred_xgb = xgb_model.predict(X_test)

# Evaluate
accuracy = accuracy_score(y_test, y_pred_xgb)
auc = roc_auc_score(y_test, y_pred_xgb)

print(f"XGBoost Accuracy: {accuracy}")
print(f"XGBoost AUC-ROC Score: {auc}")


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [16:36:36] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBoost Accuracy: 0.7398373983739838
XGBoost AUC-ROC Score: 0.6763081395348838


###Save the Preprocessing Steps (e.g., Imputation, Label Encoding, Scaling)

In [57]:
import joblib
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

# Define your preprocessor steps (adjust to your model’s preprocessing)
preprocessor = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('scaler', StandardScaler())
])

# Save preprocessor
joblib.dump(preprocessor, 'preprocessor.pkl')


['preprocessor.pkl']